# Melhorias em implementação:

In [29]:
from sgp4.api import SGP4_ERRORS

from astropy.coordinates import TEME, CartesianRepresentation  # CartesianDifferential
from astropy import units as u
from astropy.coordinates import ITRS

import pymap3d as pm

import pandas as pd
from sgp4 import omm
from sgp4.api import Satrec
from astropy.time import Time
from astropy.time import TimeDelta
import time
import numpy as np

class PropagInit:
    def __init__(self, orbital_elem, lc, sample_time=1):
        self.lc = lc
        self.sample_time = sample_time
        self.sample_fast = 30
        self.time_array = []
        self.enu = []
        self.az_el_r = []
        self.itrs = []
        self.geodetic = []
        satellite = Satrec()
        omm.initialize(satellite, orbital_elem)
        self.satellite = satellite

    def orbit_propag(self, h0, n, sample_time_v):
        time_array = np.linspace(h0,h0 + TimeDelta(sample_time_v*(n -1)* u.s),n)
        error_code, teme_p, teme_v = self.satellite.sgp4_array(time_array.jd1, time_array.jd2)
        teme_p = CartesianRepresentation(teme_p[:,0]*u.km, teme_p[:,1]*u.km, teme_p[:,2]*u.km)
        teme = TEME(teme_p, obstime=time_array)  
        itrsp = teme.transform_to(ITRS(obstime=time_array))
        location = itrsp.earth_location
        enu_p = pm.ecef2enu(u.km.to(u.m,location.x).value,
                    u.km.to(u.m,location.y).value,
                    u.km.to(u.m,location.z).value,
                    self.lc.lat, self.lc.lon, self.lc.height)
        az_el_r = np.transpose(pm.enu2aer(enu_p[0],enu_p[1],enu_p[2]))
        enu_p = np.transpose(enu_p)
        geodetic = np.transpose(location.geodetic)
        geocentric = np.transpose([location.x.value,location.y.value,location.z.value])   

        return time_array, enu_p, az_el_r, geodetic, geocentric

    def traj_calc(self, h0, n):
        time_array, enu_p, az_el_r, geodetic, geocentric = self.orbit_propag(h0, n, self.sample_time)        
  
        self.time_array.append(time_array)
        self.enu.append(enu_p)
        self.az_el_r.append(az_el_r)
        self.geodetic.append(geodetic)
        self.itrs.append(geocentric)
        return self

    def search2h0(self, t_inic, t_final, dist_max, dist_min):
        time_array_fast, enu_p, az_el_r, geodetic, geocentric = self.orbit_propag(t_inic,
                                                    1 + int((t_final-t_inic)/TimeDelta(self.sample_fast * u.s)),
                                                    self.sample_fast)                                                    
        enu_d = az_el_r[:,2]
        dist_dif = enu_d - dist_max
        idx1 = np.where(dist_dif[:-1] * dist_dif[1:] < 0 )[0] + 1      

        i = 0
        while i < len(idx1)-1:
            if dist_dif[idx1[i]]<0:
                #print( 'H0: {}, HF: {} \n'.format(time_array[idx1[i]], time_array[idx1[i+1]]))
                t_in = time_array_fast[idx1[i]] - TimeDelta(self.sample_fast * u.s)
                t_o = time_array_fast[idx1[i+1]]
                #print( 'H0: {}, HF: {} \n'.format(t_in, t_o))        
                time_array, enu_p, az_el_r, geodetic, geocentric = self.orbit_propag(t_in,
                                                            1 + int((t_o-t_in)/TimeDelta(self.sample_time * u.s)),
                                                            self.sample_time)

                #print( 'H0: {}, HF: {}, dmin: {}, d0: {}, df: {} \n'.format(time_array[0], time_array[-1], np.min(enu_d),enu_d[0],enu_d[-1] ))
                enu_d = az_el_r[:,2]
                idx = np.where(enu_d < dist_max)[0]
                time_array = time_array[idx]
                enu_p = enu_p[idx]
                az_el_r = az_el_r[idx]
                geodetic = geodetic[idx]
                geocentric = geocentric[idx]               
                
                d_min = np.min(enu_d)
                

                if d_min < dist_min:
                    self.time_array.append(time_array)
                    self.enu.append(enu_p)
                    self.az_el_r.append(az_el_r)
                    self.geodetic.append(geodetic)
                    self.itrs.append(geocentric)              
                i+=2
            else: 
                i+=1        

        return self

class LocalFrame:
    def __init__(self, lat, lon, height):
        self.lat = lat
        self.lon = lon
        self.height = height

In [26]:
orbital_elem = pd.read_csv("orbital_elem.csv").to_dict('records')
lc = LocalFrame(-5.923654, -35.167755, 50)

posit = PropagInit(orbital_elem[20],lc)

t_inic=Time('2022-08-11T00:00:00.000', format='isot', scale='utc')     # momento de inicio da busca
t_final= Time('2022-08-18T00:00:00.000', format='isot', scale='utc')   # momento de parar a busca

ini = time.time()
posit.search2h0(t_inic, t_final, 1100*1000, 1000*1000)
fim = time.time()
print("Tempo: ", fim - ini)

Tempo:  0.13399910926818848


In [30]:
tim = []
for index in range(2):
    propag = PropagInit(orbital_elem[index], lc, 1) 
    pos = propag.search2h0(t_inic, t_final, 1100*1000, 1000*1000)
    for i in range(0, len(pos.time_array)):
        time_arr = pos.time_array[i]
        tim.append(pos.time_array[i])

In [32]:
df_enu = pd.DataFrame(pos.enu[0])


In [34]:
df_enu.to_csv("TU.txt", index=False, header=['1',str(len(df_enu.index)),'1'],float_format="%.3f")

In [31]:
tim[0].value

array(['2022-08-11T02:47:39.000', '2022-08-11T02:47:40.000',
       '2022-08-11T02:47:41.000', '2022-08-11T02:47:42.000',
       '2022-08-11T02:47:43.000', '2022-08-11T02:47:44.000',
       '2022-08-11T02:47:45.000', '2022-08-11T02:47:46.000',
       '2022-08-11T02:47:47.000', '2022-08-11T02:47:48.000',
       '2022-08-11T02:47:49.000', '2022-08-11T02:47:50.000',
       '2022-08-11T02:47:51.000', '2022-08-11T02:47:52.000',
       '2022-08-11T02:47:53.000', '2022-08-11T02:47:54.000',
       '2022-08-11T02:47:55.000', '2022-08-11T02:47:56.000',
       '2022-08-11T02:47:57.000', '2022-08-11T02:47:58.000',
       '2022-08-11T02:47:59.000', '2022-08-11T02:48:00.000',
       '2022-08-11T02:48:01.000', '2022-08-11T02:48:02.000',
       '2022-08-11T02:48:03.000', '2022-08-11T02:48:04.000',
       '2022-08-11T02:48:05.000', '2022-08-11T02:48:06.000',
       '2022-08-11T02:48:07.000', '2022-08-11T02:48:08.000',
       '2022-08-11T02:48:09.000', '2022-08-11T02:48:10.000',
       '2022-08-11T02:48

In [ ]:


orbital_elem = pd.read_csv("orbital_elem.csv").to_dict('records')
lc = LocalFrame(-5.923654, -35.167755, 50)

satellite = Satrec()
omm.initialize(satellite, orbital_elem[20])

posit = PropagInit(satellite,lc)


t_inic=Time('2022-08-11T00:00:00.000', format='isot', scale='utc')     # momento de inicio da busca
t_final= Time('2022-08-18T00:00:00.000', format='isot', scale='utc')   # momento de parar a busca

ini = time.time()
posit.searchh0(t_inic, t_final, 1100*1000,  1000*1000,  10000)
fim = time.time()
print("Tempo: ", fim - ini)

In [ ]:
posit.tempo

In [4]:
orbital_elem[20]['TLE_LINE1']

'1 22015U 92039B   22223.28147519  .00006192  00000-0  22215-3 0  9994'

In [5]:
orbital_elem[20]['TLE_LINE2']

'2 22015  20.7129 189.9538 0042459  90.0946 270.4360 15.23545436676838'

## Melhoria Encontrar H0 rápido:

In [ ]:
# Para Editar:

import numpy as np
from scipy.signal import argrelextrema

orbital_elem = pd.read_csv("orbital_elem.csv").to_dict('records')
lc = LocalFrame(-5.923654, -35.167755, 50)

t_inic=Time('2022-08-11T00:00:00.000', format='isot', scale='utc')     # momento de inicio da busca
t_final= Time('2022-08-18T00:00:00.000', format='isot', scale='utc')   # momento de parar a busca

sample_fast = 30
sample_time = 1
dist_max =1100000

def calc1(orbital_elem, init, sample_time, n):   
    satellite = Satrec()
    omm.initialize(satellite, orbital_elem)
    time_array = np.linspace(init,init + TimeDelta(sample_time*(n -1)* u.s),n)
    print(time_array.value )
    error_code, teme_p, teme_v = satellite.sgp4_array(time_array.jd1, time_array.jd2)
    x, y, z = teme_p[:,0], teme_p[:,1], teme_p[:,2]
    teme_p = CartesianRepresentation(x*u.km, y*u.km, z*u.km)
    teme = TEME(teme_p, obstime=time_array)  
    itrsp = teme.transform_to(ITRS(obstime=time_array))
    location = itrsp.earth_location
    enu_p = pm.ecef2enu(1000 * location.x.value,
                        1000 * location.y.value,
                        1000 * location.z.value, lc.lat,
                                lc.lon, lc.height)

    enu_p = np.transpose(enu_p)
    enu_d = np.linalg.norm(enu_p,axis=1)
    return time_array, enu_p, enu_d

ini = time.time()

#time_array, enu_p, enu_d = calc(orbital_elem[20], t_inic, t_final, sample_fast)
time_array, enu_p, enu_d = calc1(orbital_elem[20], t_inic, sample_fast, 1 + int((t_final-t_inic)/TimeDelta(sample_fast * u.s)))


dist_dif = enu_d - dist_max # angulo de 30 no ref ENU, 0.5 = sin(30)
idx1 = np.where(dist_dif[:-1] * dist_dif[1:] < 0 )[0] +1
tim_s = time_array[idx1]
# print(dist_dif[357:363])
# print(idx1)
# print(tim_s.value)

fim = time.time()
print("Tempo: ", fim - ini)

ini = time.time()

i = 0
while i < len(idx1)-1:
    if dist_dif[idx1[i]]<0:
        #print( 'H0: {}, HF: {} \n'.format(time_array[idx1[i]], time_array[idx1[i+1]]))
        t_in = time_array[idx1[i]]
        t_o = time_array[idx1[i+1]]
        print( 'H0: {}, HF: {} \n'.format(t_in, t_o))        
        # time_array1, enu_p, enu_d = calc(orbital_elem[20],
        #                                 t_in,
        #                                 t_o,
        #                                 #Time( t_in.value, scale='utc', format='isot'),
        #                                 #Time( t_o.value, scale='utc', format='isot'),
        #                                  1) #-TimeDelta(sample_fast*u.s, scale='utc')

        time_array1, enu_p, enu_d = calc1(orbital_elem[20], t_in, sample_time, 1 + int((t_o-t_in)/TimeDelta(sample_time * u.s)))

        #print( 'H0: {}, HF: {}, dmin: {}, d0: {}, df: {} \n'.format(time_array1[0], time_array1[-1], np.min(enu_d),enu_d[0],enu_d[-1] ))
        i+=2
        break
    else: i+=1
    
#     index = argrelextrema(enu_d, np.less)
#     enu_d_min =  enu_d[index]
#     idx_d_traj = np.where(enu_d < 1100000)[0]
#     traj = enu_d[idx_d_traj]

fim = time.time()
print("Tempo: ", fim - ini)

# #print(traj)
# print(enu_d)


In [27]:
(t_o-t_in)/TimeDelta(sample_time * u.s)

<Quantity 270.>

In [23]:
print(time_array1.shape)

(604800,)


In [89]:
from skyfield.api import load, wgs84

tles1 = load.tle_file('conftle.txt')

#tles2 = load.tle_file()

sat = tles1[0]
print(sat)

ini = time.time()

ts = load.timescale()

bluffton = wgs84.latlon(-5.92256, -35.1615)
t0 = ts.utc(2022, 8, 11)
t1 = ts.utc(2022, 8, 18)
t, events = sat.find_events(bluffton, t0, t1, altitude_degrees=30.0)

fim = time.time()
print("Tempo: ", fim - ini)

ini = time.time()
for ti, event in zip(t, events):
    name = ('rise above 30°', 'culminate', 'set below 30°')[event]
    print(ti.utc_strftime('%Y-%m-%dT%H:%M:%S'), name)

    tem = Time(ti.utc_strftime('%Y-%m-%dT%H:%M:%S'), format='isot')
    enup, enud = posit.pos_calc(tem) ## Lento
    print(enud)
fim = time.time()
print("numero: ", events.shape )
print("Tempo teme to enu: ", fim - ini)

catalog #22015 epoch 2022-08-11 06:45:19 UTC
Tempo:  0.017378568649291992
2022-08-11T05:58:05 rise above 30°
895977.0351996033
2022-08-11T05:59:31 culminate
679698.8065682248
2022-08-11T06:00:58 set below 30°
910673.8295055908
2022-08-11T07:38:43 rise above 30°
920549.431502886
2022-08-11T07:40:05 culminate
732715.5971286349
2022-08-11T07:41:28 set below 30°
929805.09690544
2022-08-11T16:03:37 rise above 30°
923970.8689106569
2022-08-11T16:05:26 culminate
542792.3972932128
2022-08-11T16:07:14 set below 30°
918957.1202116479
2022-08-12T05:32:48 rise above 30°
887073.4096373181
2022-08-12T05:34:20 culminate
629083.227991606
2022-08-12T05:35:52 set below 30°
898984.6363285336
2022-08-12T07:13:43 rise above 30°
911602.8536278032
2022-08-12T07:14:54 culminate
774257.9578191183
2022-08-12T07:16:06 set below 30°
920895.2144702657
2022-08-12T15:38:22 rise above 30°
931099.4777526197
2022-08-12T15:40:14 culminate
527288.1422838406
2022-08-12T15:42:05 set below 30°
926310.1782350001
2022-08-13T0

## Melhoria Código rápido de calculo trajetória:

In [95]:


tem = Time(t.utc_strftime('%Y-%m-%dT%H:%M:%S'), format='isot')

ini = time.time()

error_code, teme_p, teme_v = satellite.sgp4_array(tem.jd1, tem.jd2)

teme_p = np.array(teme_p)
x, y, z = teme_p[:,0], teme_p[:,1], teme_p[:,2]

teme_p = CartesianRepresentation(x*u.km, y*u.km, z*u.km)
teme = TEME(teme_p, obstime=tem)  # ver diferença entre UT1 e UTC
itrsp = teme.transform_to(ITRS(obstime=tem))
location = itrsp.earth_location
enu_p = pm.ecef2enu(1000 * location.x.value, 1000 * location.y.value, 1000 * location.z.value, lc.lat,
                            lc.lon, lc.height)

fim = time.time()
print("Tempo: ", fim - ini)

enu_p

Tempo:  0.003001689910888672


(array([   25466.06337329,  -234535.48500898,  -485454.81320164,
         -878175.99713957, -1024634.30626479, -1164614.57977876,
          780143.46398059,   540860.39993516,   290103.4642783 ]),
 array([ 1216598.09923688,    51033.27412469, -1116505.87515824,
         -843261.56259651,  -250690.24273257,   343486.73060408,
          927915.67102392,  -127756.80189316, -1188335.8860551 ]),
 array([700461.93302992, 802338.92532322, 704343.79299177, 704159.42803241,
        729241.30021246, 702741.30093862, 701815.80867844, 785162.04318144,
        703506.06784671]))

### Funcionando

In [3]:
import numpy as np
import pandas as pd
from astropy.coordinates import EarthLocation, AltAz


orbital_elem = pd.read_csv("traj_data.csv").to_dict('records')

lc = LocalFrame(-5.923654, -35.167755, 50)
siding_spring = EarthLocation.from_geodetic(lc.lon*u.deg,lc.lat*u.deg,lc.height*u.m,'WGS84')

usecols = ['NORAD_CAT_ID', 'H0', 'N_PT']
df_conf = pd.read_csv("traj_data.csv",usecols=usecols)

# df_conf = pd.read_csv("traj_data.csv")

ini = time.time()

for index, row in df_conf.iterrows():
    satellite = Satrec()
    omm.initialize(satellite, orbital_elem[index])
    h0 = Time(row['H0'], format='isot')
    time_array = np.linspace(h0,h0 + TimeDelta((row['N_PT'] -1)* u.s),row['N_PT'])

    error_code, teme_p, teme_v = satellite.sgp4_array(time_array.jd1, time_array.jd2)
    teme_p = np.array(teme_p)

    teme_p = CartesianRepresentation(teme_p[:,0]*u.km, teme_p[:,1]*u.km, teme_p[:,2]*u.km)
    teme = TEME(teme_p, obstime=time_array)  
    itrsp = teme.transform_to(ITRS(obstime=time_array))
    location = itrsp.earth_location

    # alt_az = itrsp.transform_to(AltAz(obstime=time_array, location=siding_spring)) 
    
    # enu_p = pm.ecef2enu(1000*location.x.value,
    #                     1000*location.y.value,
    #                     1000*location.z.value,
    #                     lc.lat, lc.lon, lc.height)

    enu_p = pm.ecef2enu(u.km.to(u.m,location.x).value,
                        u.km.to(u.m,location.y).value,
                        u.km.to(u.m,location.z).value,
                        lc.lat, lc.lon, lc.height)

    azelr = np.transpose(pm.enu2aer(enu_p[0],enu_p[1],enu_p[2]))

    enu_p = np.transpose(enu_p)

    loc_geodetic = np.transpose(location.geodetic)

    loc_geocentric = np.transpose([location.x.value,location.y.value,location.z.value])

    # enu_d = np.linalg.norm(enu_p,axis=1)
    # min_index = np.argmin(enu_d)
    # d_min = enu_d[min_index]
    # posit_min = enu_p[min_index]
    
    #break

fim = time.time()
print("Tempo: ", fim - ini)

# print(enu_d)



Tempo:  0.1809999942779541


In [4]:
azelr

array([[3.44863440e+02, 3.86474893e+01, 1.09961233e+06],
       [3.44632518e+02, 3.88975586e+01, 1.09491980e+06],
       [3.44397962e+02, 3.91490146e+01, 1.09025339e+06],
       [3.44159693e+02, 3.94018506e+01, 1.08561346e+06],
       [3.43917630e+02, 3.96560588e+01, 1.08100035e+06],
       [3.43671691e+02, 3.99116303e+01, 1.07641441e+06],
       [3.43421790e+02, 4.01685556e+01, 1.07185600e+06],
       [3.43167840e+02, 4.04268238e+01, 1.06732546e+06],
       [3.42909752e+02, 4.06864232e+01, 1.06282318e+06],
       [3.42647434e+02, 4.09473408e+01, 1.05834950e+06],
       [3.42380792e+02, 4.12095627e+01, 1.05390480e+06],
       [3.42109729e+02, 4.14730735e+01, 1.04948946e+06],
       [3.41834146e+02, 4.17378568e+01, 1.04510384e+06],
       [3.41553942e+02, 4.20038947e+01, 1.04074833e+06],
       [3.41269012e+02, 4.22711680e+01, 1.03642332e+06],
       [3.40979251e+02, 4.25396562e+01, 1.03212920e+06],
       [3.40684548e+02, 4.28093371e+01, 1.02786634e+06],
       [3.40384792e+02, 4.30801

In [31]:
location.ellipsoid

'WGS84'

In [188]:
time_array=time_array.value.reshape(204,1)

In [187]:
enu_p.shape

(204, 3)

In [190]:
np.concatenate((time_array, enu_p), axis=1) 

array([['2022-08-11T12:18:49.000', '-224250.46328727138',
        '829005.5959448543', '686737.7470922048'],
       ['2022-08-11T12:18:50.000', '-225825.519155672',
        '821675.5698217726', '687532.87235009'],
       ['2022-08-11T12:18:51.000', '-227400.33066398837',
        '814344.6091003325', '688319.984889375'],
       ['2022-08-11T12:18:52.000', '-228974.89490022988',
        '807012.7219149172', '689099.0838196492'],
       ['2022-08-11T12:18:53.000', '-230549.20895055623',
        '799679.9164014771', '689870.1682595699'],
       ['2022-08-11T12:18:54.000', '-232123.269903042',
        '792346.2006967939', '690633.2373368682'],
       ['2022-08-11T12:18:55.000', '-233697.074845232',
        '785011.5829385936', '691388.2901883955'],
       ['2022-08-11T12:18:56.000', '-235270.6208648493',
        '777676.0712661771', '692135.3259600131'],
       ['2022-08-11T12:18:57.000', '-236843.90505050047',
        '770339.6738197859', '692874.3438066747'],
       ['2022-08-11T12:18:58.

In [180]:
u.km.to(u.m,location.x).value

array([5685481.7283911 , 5684602.65866297, 5683717.13565711,
       5682825.16101302, 5681926.73637877, 5681021.86340875,
       5680110.54376518, 5679192.77911761, 5678268.57114257,
       5677337.92152428, 5676400.83195409, 5675457.3041309 ,
       5674507.33976051, 5673550.94055649, 5672588.10823927,
       5671618.84453685, 5670643.15118488, 5669661.0299257 ,
       5668672.48250906, 5667677.51069231, 5666676.11623993,
       5665668.30092384, 5664654.06652285, 5663633.4148236 ,
       5662606.3476195 , 5661572.8667116 , 5660532.97390813,
       5659486.6710246 , 5658433.95988393, 5657374.84231589,
       5656309.3201581 , 5655237.39525489, 5654159.06945828,
       5653074.34462748, 5651983.22262854, 5650885.70533561,
       5649781.79462935, 5648671.4923978 , 5647554.8005365 ,
       5646431.72094811, 5645302.25554256, 5644166.40623711,
       5643024.17495605, 5641875.56363088, 5640720.5742006 ,
       5639559.20861131, 5638391.46881648, 5637217.35677641,
       5636036.87445904,

In [181]:
loc_geocentric.shape

(204, 3)

In [143]:
pd.DataFrame(loc_geocentric,  columns=['X(km)','Y(km)','Z(km)'])

,X(km),Y(km),Z(km)
0,5394.205772,-4537.443947,-38.571297
1,5393.561744,-4538.156591,-46.056554
2,5392.911495,-4538.863994,-53.541758
3,5392.255026,-4539.566154,-61.026901
4,5391.592339,-4540.263068,-68.511975
...,...,...,...
156,5219.642891,-4584.344872,-1200.337757
157,5218.051286,-4584.223292,-1207.711725
158,5216.453785,-4584.096282,-1215.084314
159,5214.850389,-4583.963842,-1222.455516


In [125]:
pd.DataFrame(loc_geodetic, columns=['lon(deg)','lat(deg)','height(km)'])

,lon(deg),lat(deg),height(km)
0,-40.069539,-0.315431,670.788470
1,-40.077342,-0.376645,670.799634
2,-40.085145,-0.437858,670.810790
3,-40.092948,-0.499072,670.821938
4,-40.100751,-0.560285,670.833079
...,...,...,...
156,-41.292419,-9.861615,672.428877
157,-41.300330,-9.922783,672.438639
158,-41.308242,-9.983951,672.448390
159,-41.316155,-10.045119,672.458129


In [91]:
enu_df = pd.DataFrame(enu_p, columns = ['ENU_E(m)','ENU_N(m)','ENU_U(m)'])

enu_df

,ENU_E(m),ENU_N(m),ENU_U(m)
0,-602307.176699,682051.487136,611559.310061
1,-603260.684078,674594.226802,612216.422654
2,-604213.489698,667136.182382,612865.468310
3,-605165.591299,659677.362412,613506.446427
4,-606116.986621,652217.775428,614139.356411
...,...,...,...
156,-741190.630480,-485450.660314,616388.897055
157,-742007.962309,-492926.757181,615786.120227
158,-742824.252501,-500402.302530,615175.296175
159,-743639.498975,-507877.287815,614556.425761


In [6]:
aer_df = pd.DataFrame(azelr, columns = ['AZ(deg)','ELEV(deg)','RANGE(m)'])
aer_df


,AZ(deg),ELEV(deg),RANGE(m)
0,318.552866,33.904917,1.096345e+06
1,318.195099,34.078046,1.092616e+06
2,317.833418,34.250982,1.088921e+06
3,317.467787,34.423692,1.085260e+06
4,317.098170,34.596141,1.081634e+06
...,...,...,...
156,236.776781,34.825768,1.079334e+06
157,236.403330,34.654592,1.082934e+06
158,236.033888,34.483120,1.086568e+06
159,235.668421,34.311387,1.090238e+06


In [114]:
conca = enu_df.join( aer_df)
conca

,ENU_E(m),ENU_N(m),ENU_U(m),AZ(deg),ELEV(deg),RANGE(m)
0,-602307.176699,682051.487136,611559.310061,318.552866,33.904917,1.096345e+06
1,-603260.684078,674594.226802,612216.422654,318.195099,34.078046,1.092616e+06
2,-604213.489698,667136.182382,612865.468310,317.833418,34.250982,1.088921e+06
3,-605165.591299,659677.362412,613506.446427,317.467787,34.423692,1.085260e+06
4,-606116.986621,652217.775428,614139.356411,317.098170,34.596141,1.081634e+06
...,...,...,...,...,...,...
156,-741190.630480,-485450.660314,616388.897055,236.776781,34.825768,1.079334e+06
157,-742007.962309,-492926.757181,615786.120227,236.403330,34.654592,1.082934e+06
158,-742824.252501,-500402.302530,615175.296175,236.033888,34.483120,1.086568e+06
159,-743639.498975,-507877.287815,614556.425761,235.668421,34.311387,1.090238e+06


In [87]:
location.value[0]


(5394.20577168, -4537.44394659, -38.57129728)

In [41]:
azelr[0]

array([3.18552866e+02, 3.39049174e+01, 1.09634527e+06])

In [89]:
alt_az[0]

<AltAz Coordinate (obstime=2022-08-11T15:00:54.000, location=(5186315.09144039, -3654173.38027844, -653865.96464262) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt, distance) in (deg, deg, km)
    (318.57173307, 33.92144438, 1095.90569378)>

In [263]:
np.shape(enu_p)

(233, 3)

In [151]:
import numpy as np
orbital_elem = pd.read_csv("traj_data.csv").to_dict('records')
lc = LocalFrame(-5.923654, -35.167755, 50)

usecols = ['NORAD_CAT_ID', 'H0', 'N_PT']
df_conf = pd.read_csv("traj_data.csv",usecols=usecols)

# df_conf = pd.read_csv("traj_data.csv")

ini = time.time()

for index, row in df_conf.iterrows():
    satellite = Satrec()
    orbital_elem_row = next(x for x in orbital_elem if x["NORAD_CAT_ID"] == row['NORAD_CAT_ID'])
    omm.initialize(satellite, orbital_elem_row)
    propag = PropagInit(satellite, lc, 1) 
    pos = propag.orbitpropag(Time(row['H0'], format='isot'),row['N_PT'])
    
    break

fim = time.time()
print("Tempo: ", fim - ini)


NORAD: 2142 dmin= 776821.976
Tempo:  0.39698338508605957


In [232]:
import numpy as np
orbital_elem = pd.read_csv("traj_data.csv").to_dict('records')
lc = LocalFrame(-5.923654, -35.167755, 50)

usecols = ['NORAD_CAT_ID', 'H0', 'N_PT']
df_conf = pd.read_csv("traj_data.csv",usecols=usecols)

# df_conf = pd.read_csv("traj_data.csv")

ini = time.time()
sdf = SummarizeDataFiles()
for index, row in df_conf.iterrows():
    satellite = Satrec()
    orbital_elem_row = next(x for x in orbital_elem if x["NORAD_CAT_ID"] == row['NORAD_CAT_ID'])
    omm.initialize(satellite, orbital_elem_row)
    propag = PropagInit(satellite, lc, 1) 
    pos = propag.orbit_propag(Time(row['H0'], format='isot'),row['N_PT'])
    sdf.save_trajectories(pos,orbital_elem_row)
    break

fim = time.time()
print("Tempo: ", fim - ini)

Tempo:  0.0039997100830078125


In [200]:
np.shape(pos.dist )


(1, 3)

In [231]:
class SummarizeDataFiles:
    def __init__(self):
        self.sel_orbital_elem = []
        self.sel_resume = { "H0":[], "DIST_H0":[],"H_DIST_MIN":[],"PT_DIST_MIN":[],
                    "DIST_MIN":[],"HF":[], "N_PT":[], "DIST_HF":[],"RCS":[] }

    def save_trajectories(self,pos,orbital_elem): 
        for i in range(0, len(pos.traj)):
            tempo = pos.tempo[i]
            posenu = pos.traj[i]
            distenu = pos.dist[i]
            
            ttxt = tempo[0].strftime('%Y_%m_%d-H0-%H_%M_%S')

            # Summarized data set of the trajectories obtained
            self.sel_orbital_elem.append(orbital_elem) 

            self.sel_resume["RCS"].append(orbital_elem['RCS_SIZE'])
            self.sel_resume["H0"].append(tempo[0].value)
            self.sel_resume["DIST_H0"].append(distenu[0])
            self.sel_resume["H_DIST_MIN"].append(pos.hdmin[i].value[11:])
            self.sel_resume["PT_DIST_MIN"].append(pos.hrmin[i])
            self.sel_resume["DIST_MIN"].append(pos.dmin[i])
            self.sel_resume["HF"].append(tempo[len(tempo) - 1].value[11:])
            self.sel_resume["N_PT"].append(len(distenu) - 1)
            self.sel_resume["DIST_HF"].append(distenu[len(distenu) - 1]) 

In [233]:
sdf.sel_resume

{'H0': ['2022-08-06T12:30:39.000'],
 'DIST_H0': [1070257.5070111575],
 'H_DIST_MIN': ['12:32:39.000'],
 'PT_DIST_MIN': [120],
 'DIST_MIN': [11266.620239357559],
 'HF': ['12:34:31.000'],
 'N_PT': [232],
 'DIST_HF': [1261516.725177206],
 'RCS': ['LARGE']}

In [56]:
norad_id = pd.read_csv("orbital_elem1.csv")
norad_id_c = norad_id.drop_duplicates(subset=['NORAD_CAT_ID'])

In [57]:
norad_id.shape

(68, 40)

In [58]:
norad_id_c.shape

(50, 40)

## Procura automática de H0 Backup

In [ ]:
# Para Editar:

import numpy as np
from scipy.signal import argrelextrema

orbital_elem = pd.read_csv("orbital_elem.csv").to_dict('records')
lc = LocalFrame(-5.923654, -35.167755, 50)

t_inic=Time('2022-08-11T00:00:00.000', format='isot', scale='utc')     # momento de inicio da busca
t_final= Time('2022-08-18T00:00:00.000', format='isot', scale='utc')   # momento de parar a busca

sample_fast = 60
dist_max =1100000

def calc(orbital_elem, init,end, sample_time):    
    satellite = Satrec()
    omm.initialize(satellite, orbital_elem)
    pts = (end -init)/TimeDelta(sample_time * u.s)
    time_array = np.linspace(init, end ,int(pts.value)+1)
    print(time_array.shape )
    error_code, teme_p, teme_v = satellite.sgp4_array(time_array.jd1, time_array.jd2)
    x, y, z = teme_p[:,0], teme_p[:,1], teme_p[:,2]
    teme_p = CartesianRepresentation(x*u.km, y*u.km, z*u.km)
    teme = TEME(teme_p, obstime=time_array)  
    itrsp = teme.transform_to(ITRS(obstime=time_array))
    location = itrsp.earth_location
    enu_p = pm.ecef2enu(1000 * location.x.value, 1000 * location.y.value, 1000 * location.z.value, lc.lat,
                                lc.lon, lc.height)
    
    enu_p = np.transpose(enu_p) 
    enu_d = np.linalg.norm(enu_p,axis=1)
    #print(enu_d )
    return time_array, enu_p, enu_d

ini = time.time()

time_array, enu_p, enu_d = calc(orbital_elem[20], t_inic, t_final, sample_fast)

# satellite = Satrec()
# omm.initialize(satellite, orbital_elem[20])


# time_array = np.linspace(t_inic,t_final,int(pts.value)+1 )
# #print(time_array.value )
# error_code, teme_p, teme_v = satellite.sgp4_array(time_array.jd1, time_array.jd2)
# x, y, z = teme_p[:,0], teme_p[:,1], teme_p[:,2]
# teme_p = CartesianRepresentation(x*u.km, y*u.km, z*u.km)
# teme = TEME(teme_p, obstime=time_array)  
# itrsp = teme.transform_to(ITRS(obstime=time_array))
# location = itrsp.earth_location
# enu_p = pm.ecef2enu(1000 * location.x.value, 1000 * location.y.value, 1000 * location.z.value, lc.lat,
#                              lc.lon, lc.height)

#enu_p = np.transpose(enu_p)
# idx_z_pos = np.where(enu_p[:,2] > 0)[0]
# #print(idx_z_pos)
# enu_p_z = enu_p[idx_z_pos]
# #print(enu_p_z)
# enu_d = np.linalg.norm(enu_p_z,axis=1)
# idx_d = np.where(enu_d < 5000000)[0]
# #print(enu_d[idx_d])
# enu_d5 = enu_d[idx_d]
# index = argrelextrema(enu_d5, np.less)
# print(enu_d[index])

# enu_p = np.transpose(enu_p) 
# enu_d = np.linalg.norm(enu_p,axis=1)
# #print(enu_d)
# index = argrelextrema(enu_d, np.less)
# enu_d_min =  enu_d[index]
# idx_d_val = np.where(enu_d_min < 1100000)[0]

# enu_p = np.transpose(enu_p) 
# enu_d = np.linalg.norm(enu_p,axis=1)
# index = argrelextrema(enu_d, np.less)
# enu_d_min =  enu_d[index]
# tim_s = time_array[index]
# idx_d_val = np.where((enu_d_min < 1*1100000)&(enu_d_min >0) )[0]

# enu_p = np.transpose(enu_p) 
# enu_d = np.linalg.norm(enu_p,axis=1)
# a = enu_p[:,2] - 0.5*enu_d # angulo de 30 no ref ENU, 0.5 = sin(30)
# idx1 = np.where(a[:-1] * a[1:] < 0 )[0] +1
# tim_s = time_array[idx1]
# print(a[358:362])
# print(idx1)

dist_dif = enu_d - dist_max # angulo de 30 no ref ENU, 0.5 = sin(30)
idx1 = np.where(dist_dif[:-1] * dist_dif[1:] < 0 )[0] +1
tim_s = time_array[idx1]
# print(dist_dif[357:363])
# print(idx1)
# print(tim_s.value)

# index_min = argrelextrema(enu_p[:,2], np.greater)
# index_in_ot = np.where((np.abs(enu_p[:,2]) < 1*1100000)&(enu_d_min >0) )[0]
# tim_s = time_array[index]
# idx_d_val = np.where((enu_d_min < 1*1100000)&(enu_d_min >0) )[0]

# time_selec = tim_s[idx_d_val]
fim = time.time()
print("Tempo: ", fim - ini)

ini = time.time()

i = 0
while i < len(idx1)-1:
    if dist_dif[idx1[i]]<0:
        #print( 'H0: {}, HF: {}, d: {} \n'.format(time_array[idx1[i]], time_array[idx1[i+1]], enu_d[idx1[i]] ))
        time_array1, enu_p, enu_d = calc(orbital_elem[20],time_array[idx1[i]],time_array[idx1[i+1]], 1)
        print( 'H0: {}, HF: {}, d: {} \n'.format(time_array1[0], time_array1[-1], np.min(enu_d) ))
        i+=2
    else: i+=1
    
#     enu_p = np.transpose(enu_p) 
#     enu_d = np.linalg.norm(enu_p,axis=1)
#     index = argrelextrema(enu_d, np.less)
#     enu_d_min =  enu_d[index]
#     idx_d_traj = np.where(enu_d < 1100000)[0]
#     traj = enu_d[idx_d_traj]

fim = time.time()
print("Tempo: ", fim - ini)

# #print(traj)
# print(enu_d)

In [ ]:
            df_conf = pd.DataFrame(np.concatenate((lc.lat,lc.lon,lc.height, dmax,dmin, sample_time ), axis=1),
                                    columns=[ 'POINT_LAT(deg)','POINT_LON(deg)','POINT_HEIG(m)',
                                        'MAX_RANGE(Km)','MIN_HANGE(Km)','POINT_HEIG(m)','SAMPLE_TIME(s)'])
            df_conf.to_csv(dir_name + "/"+ date_time[0:-4] +"_conf_summary.csv", index=False) 

In [10]:
import pandas as pd

df = pd.read_csv('https://celestrak.org/NORAD/elements/gp.php?CATNR=25544&FORMAT=csv')

df

,OBJECT_NAME,OBJECT_ID,EPOCH,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,EPHEMERIS_TYPE,CLASSIFICATION_TYPE,NORAD_CAT_ID,ELEMENT_SET_NO,REV_AT_EPOCH,BSTAR,MEAN_MOTION_DOT,MEAN_MOTION_DDOT
0,ISS (ZARYA),1998-067A,2022-08-17T20:17:15.171072,15.502142,0.000553,51.643,31.1298,133.1079,8.5802,0,U,25544,999,35474,0.000105,0.000056,0


In [7]:
df

,No GP data found


In [11]:
xx = { "H0":0, "RANGE_H0":0,"MIN_RANGE_H":0}

In [20]:
rcs = pd.read_csv('RCS.csv').to_dict('list')

In [23]:
rcs['RCS'][rcs['NORAD_CAT_ID'].index(25544)]

399.0524

In [19]:
import pandas as pd
satcat_df = pd.read_csv('https://celestrak.org/pub/satcat.csv')  

In [20]:
satcat_df.head()

,OBJECT_NAME,OBJECT_ID,NORAD_CAT_ID,OBJECT_TYPE,OPS_STATUS_CODE,OWNER,LAUNCH_DATE,LAUNCH_SITE,DECAY_DATE,PERIOD,INCLINATION,APOGEE,PERIGEE,RCS,DATA_STATUS_CODE,ORBIT_CENTER,ORBIT_TYPE
0,SL-1 R/B,1957-001A,1,R/B,D,CIS,1957-10-04,TYMSC,1957-12-01,96.19,65.10,938.0,214.0,20.420,NaN,EA,IMP
1,SPUTNIK 1,1957-001B,2,PAY,D,CIS,1957-10-04,TYMSC,1958-01-03,96.10,65.00,1080.0,64.0,NaN,NaN,EA,IMP
2,SPUTNIK 2,1957-002A,3,PAY,D,CIS,1957-11-03,TYMSC,1958-04-14,103.74,65.33,1659.0,211.0,0.080,NaN,EA,IMP
3,EXPLORER 1,1958-001A,4,PAY,D,US,1958-02-01,AFETR,1970-03-31,88.48,33.15,215.0,183.0,NaN,NaN,EA,IMP
4,VANGUARD 1,1958-002B,5,PAY,NaN,US,1958-03-17,AFETR,NaN,132.72,34.26,3833.0,648.0,0.122,NaN,EA,ORB


In [21]:
# df.drop(df.loc[df['line_race']==0].index, inplace=True)
from asyncio.windows_events import NULL
from cmath import nan


satcat_df.drop(satcat_df.loc[satcat_df['DECAY_DATE']!=NULL].index, inplace=True)

In [22]:
satcat_df

,OBJECT_NAME,OBJECT_ID,NORAD_CAT_ID,OBJECT_TYPE,OPS_STATUS_CODE,OWNER,LAUNCH_DATE,LAUNCH_SITE,DECAY_DATE,PERIOD,INCLINATION,APOGEE,PERIGEE,RCS,DATA_STATUS_CODE,ORBIT_CENTER,ORBIT_TYPE


In [28]:
from spacetrack import SpaceTrackClient
from io import StringIO

with open('secret.txt') as file:
    passw = file.read()

stc = SpaceTrackClient(identity='francisval20@yahoo.com.br', password=passw)    
try:
    stc.authenticate()
except:
    log_error = '<p style="font-family:sans-serif; color:Red; font-size: 16px;">Space-Track Loguin Error</p>'
    print('space-track loguin error')
#tlevec_csv = stc.gp(norad_cat_id=norad_ids, orderby='norad_cat_id', format='csv')
# tlevec_csv = stc.omm(norad_cat_id=25544, orderby='epoch desc', limit=22, format='csv')
# elem_df = pd.read_csv(StringIO(tlevec_csv), sep=",")


In [ ]:
tlevec_csv = stc.omm(norad_cat_id=25544, orderby='epoch desc', limit=22, format='csv')
elem_df = pd.read_csv(StringIO(tlevec_csv), sep=",")

In [32]:
import spacetrack.operators as op
tlevec_csv = stc.satcat(period=op.less_than(128), decay=None, current='Y',format='csv')
elem_df = pd.read_csv(StringIO(tlevec_csv), sep=",")

In [33]:
elem_df.shape

(20036, 24)

In [ ]:
elem_df.columns.to_list()